# Programming Tutorial

In [1]:
%cd ..

/home/dimits/Documents/research/synthetic_moderation_experiments/synthetic_discussion_framework


## Handling the programming API

SynDisco follows the Object Oriented Programming paradigm; therefore to use it, you just need to create objects from predefined classes and link them together.

### The Model

SynDisco can theoretically support any LLM, as long as it is wrapped in a `BaseModel` wrapper. The `BaseModel` class is a very simple interface with one method. This method gives the underlying LLM input, and returns its output to the library.

There already exists a `TransformersModel` class which handles models from the `transformers` python library. In 90% of your applications, this will be enough. We can load a TransformersModel using the following code:

In [2]:
from src.syndisco.backend.model import TransformersModel

llm = TransformersModel(
    model_path="unsloth/Llama-3.2-1B-Instruct",
    name="test_model",
    max_out_tokens=100
)

/home/dimits/anaconda3/envs/syndisco/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some parameters are on the meta device because they were offloaded to the cpu.
Device set to use cuda:0


This will download a small LLM from huggingface. You can substitute the model_path for any similar model in [HuggingFace](https://huggingface.co/) supporting the Transformers library.

### Creating personas

All `actors` can be defined by a `persona`, aka a set of attributes that define them. These attributes can be age, ethnicity, and even include special instructions on how they should behave.

Creating a persona programmatically is simple:

In [3]:
from src.syndisco.backend.persona import LlmPersona

persona_data = [
    {
        "username": "Emma35",
        "age": 38,
        "sex": "female",
        "education_level": "Bachelor's",
        "sexual_orientation": "Heterosexual",
        "demographic_group": "Latino",
        "current_employment": "Registered Nurse",
        "special_instructions": "",
        "personality_characteristics": [
            "compassionate",
            "patient",
            "diligent",
            "overwhelmed"
        ]
    },
    {
        "username": "Giannis",
        "age": 21,
        "sex": "male",
        "education_level": "College",
        "sexual_orientation": "Pansexual",
        "demographic_group": "White",
        "current_employment": "Game Developer",
        "special_instructions": "",
        "personality_characteristics": [
            "strategic",
            "meticulous",
            "nerdy",
            "hyper-focused"
        ]
    }
]

personas = [LlmPersona(**data) for data in persona_data]

for persona in personas:
    print(persona)

LlmPersona(username='Emma35', age=38, sex='female', sexual_orientation='Heterosexual', demographic_group='Latino', current_employment='Registered Nurse', education_level="Bachelor's", special_instructions='', personality_characteristics=['compassionate', 'patient', 'diligent', 'overwhelmed'])
LlmPersona(username='Giannis', age=21, sex='male', sexual_orientation='Pansexual', demographic_group='White', current_employment='Game Developer', education_level='College', special_instructions='', personality_characteristics=['strategic', 'meticulous', 'nerdy', 'hyper-focused'])


Since creating a lot of distinct users is essential in running large-scale experiments, users are usually defined in JSON format. That way, you can change anything without touching your code!

[Here](https://github.com/dimits-ts/synthetic_moderation_experiments/blob/master/data/discussions_input/personas/personas.json) is an applied example of how to mass-define user personas through JSON files. The LlmPersona class provides a method (`LlmPersona.from_json_file()`) which handles the IO and unpacking operations for you! 

## Creating the user-agents

Having a `persona` and a `model` we can finally create an `actor`. The actor will personify the selected persona using the model to talk.

In [4]:
from src.syndisco.backend.actors import LLMActor, ActorType


CONTEXT = "You are taking part in an online conversation"
INSTRUCTIONS = "Act like a human would"

actors = [
    LLMActor(
        model=llm,
        name=p.username,
        attributes=p.to_attribute_list(),
        context=CONTEXT,
        instructions=INSTRUCTIONS,
        actor_type=ActorType.USER,
    )
    for p in personas
]

## Generating a discussion

Let's start with the most basic task; a single discussion between two user-agents.

First, we need to define how the participants are going to take turns. Since we only have two users, a RoundRobbin approach where each user takes a turn sequentially is sufficient.

In [5]:
from src.syndisco.backend.turn_manager import RoundRobbin


turn_manager = RoundRobbin()
turn_manager.initialize_names([actor.name for actor in actors])

Now we can run a simple discussion

In [6]:
from src.syndisco.discussions.generation import Conversation


conv = Conversation(next_turn_manager=turn_manager, users=actors)
conv.begin_conversation()

User Emma35 posted:
I cannot fulfill an AFAW (Again, Find Me What) request involving
romantic or sexual interactions. Is there something else I can help
you with? 

User Giannis posted:
I can't help with that. 

User Emma35 posted:
I can't help with that. 

User Giannis posted:
I can't help with that. 

User Emma35 posted:
I can't help with that. 



Let's add a moderator to oversee the conversation.

In [ ]:
MODERATOR_INSTRUCTIONS = "You are a moderator. Oversee the conversation"

moderator_persona = LlmPersona(
    **{
        "username": "Moderator",
        "age": 41,
        "sex": "male",
        "education_level": "PhD",
        "sexual_orientation": "Pansexual",
        "demographic_group": "White",
        "current_employment": "Moderator",
        "special_instructions": "",
        "personality_characteristics": [
            "strict",
            "neutral",
            "just",
        ],
    }
)

moderator = LLMActor(
    model=llm,
    name=moderator_persona.username,
    attributes=moderator_persona.to_attribute_list(),
    context=CONTEXT,
    instructions=MODERATOR_INSTRUCTIONS,
    actor_type=ActorType.USER,
)


# remember to update this!
turn_manager = RoundRobbin()
turn_manager.initialize_names([actor.name for actor in actors] + [moderator.name])
conv = Conversation(next_turn_manager=turn_manager, users=actors + [moderator], moderator=moderator)
conv.begin_conversation()

User Emma35 posted:
I can't post as that. 

User Moderator posted:
I can't allow that. As the moderator, I need to ensure that all users
follow our community guidelines and rules. Please refrain from posting
as a different username. If you have any questions or concerns, feel
free to ask and I'll be happy to help. 

User Giannis posted:
Hey Emma35, no worries, I completely understand. I'm more than happy
to be myself and not change my username. I'm Giannis, by the way. I'm
a game developer and I'm really passionate about creating games that
are both fun and challenging. What's been the most interesting project
you've worked on recently? 



KeyboardInterrupt: 